In [1]:
"""
import ibmos2spark
# @hidden_cell
credentials = {
    'endpoint': 'https://s3.eu-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-1d71bac5-a341-449f-9ca7-b29de95e771a',
    'iam_service_endpoint': 'https://iam.eu-gb.bluemix.net/oidc/token',
    'api_key': 'ejujKveBRS3Bk7l3pzjemEQDhTSaOKmqCg6x6osXWkx4'
}

configuration_name = 'os_2d970470a6354234a1716fe9f4db519b_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

df = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('US_Accidents_Dec19.csv', 'bdppproject-donotdelete-pr-nejualq57kuqe4'))
"""

"\nimport ibmos2spark\n# @hidden_cell\ncredentials = {\n    'endpoint': 'https://s3.eu-geo.objectstorage.service.networklayer.com',\n    'service_id': 'iam-ServiceId-1d71bac5-a341-449f-9ca7-b29de95e771a',\n    'iam_service_endpoint': 'https://iam.eu-gb.bluemix.net/oidc/token',\n    'api_key': 'ejujKveBRS3Bk7l3pzjemEQDhTSaOKmqCg6x6osXWkx4'\n}\n\nconfiguration_name = 'os_2d970470a6354234a1716fe9f4db519b_configs'\ncos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')\n\nfrom pyspark.sql import SparkSession\nspark = SparkSession.builder.getOrCreate()\n\ndf = spark.read  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')  .option('header', 'true')  .load(cos.url('US_Accidents_Dec19.csv', 'bdppproject-donotdelete-pr-nejualq57kuqe4'))\n"

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

try:
    spark.close()
except: 
    pass

try:
    conf = SparkConf().setAppName('Spark Project')
    sc = SparkContext(conf=conf)
except Exception as e:
    print(e)
#.config("spark.some.config.option", "some-value") \
#SparkContext.setSystemProperty('spark.executor.memory', '8g')
#spark = SparkSession \
#    .builder \
#    .appName("Spark Project") \
#    .config("spark.executor.memory", '5g') \
#    .config("spark.driver.memory", '5g') \
#    .getOrCreate()

spark = SparkSession.builder \
    .appName("Spark Project") \
    .config("spark.executor.memory", "2000mb") \
    .master("local[*]") \
    .config("spark.executor.cores", "4") \
    .config("spark.yarn.executor.memoryOverhead",200) \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.default.parallelism", "6") \
    .getOrCreate()

df = spark.read.load("data/US_Accidents_Dec19.csv",format="csv", sep=",", inferSchema="true", header="true")

In [ ]:
sc

In [ ]:
sc._conf.getAll()

# Imports

In [ ]:
from pyspark.sql.functions import when, count, col, isnan, countDistinct,from_unixtime,from_utc_timestamp, unix_timestamp,split, to_timestamp, hour, month, lit,collect_list
import seaborn as sns 
import matplotlib.pyplot as plt 

# Due to problem in spark 3.0.0
spark.conf.set("spark.sql.legacy.utcTimestampFunc.enabled", "true")

# Define variables

In [ ]:
# Define data information

# Numerical values

colLabel = ["Severity"]

colRem = ['ID', 
          'Source',
          'End_Time',
          'End_Lat',
          'End_Lng',
          'Description',
        ]

# Dropping data that cant help during model
df = df.drop(*colRem)

# Convert boolean to string since PCA cant handle boolean which should be a class
df = df.select(*[col(c[0]).cast("string").alias(c[0]) if c[1] == 'boolean' else col(c[0]).alias(c[0]) for c in df.dtypes])

#renamedHousing.select([count(when(col(c).isNull(), c)).alias(c) for c in colNum]).show()
rdd = sc.parallelize(df.dtypes)

colCat = rdd.map(lambda i: i[0] if (i[1]=='string' or i[1]=='boolean' and i[0]) else None).filter(lambda i: i != None).collect()
colNum = rdd.map(lambda i: i[0] if (i[1]=='double' and i[0]) else None).filter(lambda i: i != None).collect()

print(colCat)
print(colNum)

# 1. Analysis

In [ ]:
print(f"Categories:\n \
Labels: {len(colLabel)}\n \
Classes: {len(colCat)}\n \
Removed: {len(colRem)}\n \
Numerical: {len(colNum)}") 

print(f"Rows: {df.count()}\nColumns {len(df.columns)}")
df.printSchema()
df.take(1)

## Statistical summary

In [ ]:
df.describe(colNum).show()

## Get missing data

In [ ]:
# Check data

df_missingVals_cols = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])

# Missing value in each column
df_missingVals_cols.collect()

## Get categorical values

In [ ]:
# Checking how many classes that can be used
df.agg(*(countDistinct(col(c)).alias(c) for c in colCat)).collect()[0].asDict()

## Correlation

In [ ]:
print([*colNum, *colLabel])

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation

assembler = VectorAssembler(
    inputCols=colNum,
    outputCol="features",
    handleInvalid = "skip")

df_attributes = assembler.transform(df)
df_attributes.select("features").show(1,False)



r1 = Correlation.corr(df_attributes, "features").head()

print("correlation matrix:\n" + str(r1[0]))

In [ ]:
sns.set(style="white")

# Generate a large random dataset

# Compute the correlation matrix

# Generate a mask for the upper triangle
#mask = np.triu(np.ones_like(r1, dtype=np.bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(r1[0].toArray().tolist(), annot=True, cmap=cmap, vmax=.3, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5},xticklabels=colNum,yticklabels=colNum)

## Severity

In [ ]:
severity_freq = df.groupBy('Severity').count().orderBy('count',ascending=False).collect()

In [ ]:
rdd = sc.parallelize(severity_freq)
pd_severity = rdd.toDF().toPandas()

# Plot data
fig,ax = plt.subplots(figsize=(16,10))
b = sns.barplot(pd_severity['Severity'],pd_severity['count'], color='blue')
b.axes.set_title("Severity distribution",fontsize=20)
b.set_xlabel("Number of Accidents",fontsize=15)
b.set_ylabel("Severity",fontsize=15)
b.tick_params(labelsize=10)

## Causes

### Frequency of severity and state

In [ ]:
state_freq = df.groupBy('State').count().orderBy('count',ascending=False).collect()

In [ ]:
rdd = sc.parallelize(state_freq)
pd_states = rdd.toDF().toPandas()

# Plot data
fig,ax = plt.subplots(figsize=(16,10))
b = sns.barplot(pd_states['State'],pd_states['count'], color='blue')
b.axes.set_title("Severity distribution for each state",fontsize=20)
b.set_xlabel("Number of Accidents",fontsize=15)
b.set_ylabel("State",fontsize=15)
b.tick_params(labelsize=10)

In [ ]:
state_severity_freq = df.groupBy('State','Severity').count().orderBy('count',ascending=False).collect()

In [ ]:
rdd = sc.parallelize(state_severity_freq)
pd_state_severity = rdd.toDF().toPandas()

# Plot data
fig,ax = plt.subplots(figsize=(16,10))
b = sns.barplot(x="State", y="count", hue="Severity", data=pd_state_severity)
#b = sns.barplot(pd_state_severity['State', 'Severity'],pd_state_severity['count'])
b.axes.set_title("Severity distribution for each state",fontsize=20)
b.set_xlabel("Number of Accidents",fontsize=15)
b.set_ylabel("State",fontsize=15)
b.tick_params(labelsize=10)

### Weather conditions

In [ ]:
weather_freq = df.groupBy('Weather_Condition').count().orderBy('count',ascending=False).collect()

In [ ]:
#display_limit = 12 # Limit on how many objects that should be displayed

fig, ax=plt.subplots(figsize=(16,25))
n = int(df.count()*0.001) # Limit the plot to ignore conditions below an limit

rdd = sc.parallelize(weather_freq)
rdd_filtered = rdd.filter(lambda x: x['count'] > n)
print(rdd_filtered.collect())


pd_weather = rdd_filtered.toDF().toPandas()

b = sns.barplot(pd_weather['count'][:],pd_weather['Weather_Condition'][:], color="blue")

b.axes.set_title("Weather Condition for accidents above 1% of total set",fontsize=20)
b.set_xlabel("Number of Accidents",fontsize=15)
b.set_ylabel("Weather_Condition",fontsize=15)
b.tick_params(labelsize=10)
plt.show()


In [ ]:
# How severe was the condition for each type of weather?

### Time when accidents occured

In [ ]:
df_time = df.selectExpr("hour(to_timestamp(from_utc_timestamp(Start_Time, Timezone), 'yyyy-MM-dd HH:mm:ss')) as Start_Time")
time_freq = df_time.groupBy('Start_Time').count().orderBy('count',ascending=False).collect()

In [ ]:
rdd = sc.parallelize(time_freq)
pd_time = rdd.toDF().toPandas()

# Plot data
fig,ax = plt.subplots(figsize=(16,10))

b = sns.barplot(pd_time['Start_Time'],pd_time['count'], color='blue')
b.axes.set_title("Daytime for accidents",fontsize=20)
b.set_xlabel("Time (Hours)",fontsize=15)
b.set_ylabel("Number of Accidents",fontsize=15)
b.tick_params(labelsize=10)
plt.show()


### Development of accidents on Month basis

In [ ]:
df_time = df.selectExpr("month(to_timestamp(from_utc_timestamp(Start_Time, Timezone), 'yyyy-MM-dd HH:mm:ss')) as Start_Time")
time_freq = df_time.groupBy('Start_Time').count().orderBy('count',ascending=False).collect()

In [ ]:
rdd = sc.parallelize(time_freq)
pd_time = rdd.toDF().toPandas()

# Plot data
fig,ax = plt.subplots(figsize=(16,10))

b = sns.barplot(pd_time['Start_Time'],pd_time['count'], color='blue')
b.axes.set_title("Montly distribution for accidents",fontsize=20)
b.set_xlabel("Time (Month)",fontsize=15)
b.set_ylabel("Number of Accidents",fontsize=15)
b.tick_params(labelsize=10)
plt.show()


### Map distribution

In [ ]:
sns.jointplot(x=df.select(collect_list('Start_Lat')).first()[0],y=df.select(collect_list('Start_Lng')).first()[0],height=10)
plt.ylabel('Start_Lat', fontsize=12)
plt.xlabel('Start_Lng', fontsize=12)
plt.show()

In [ ]:
from pyspark.ml.feature import PCA
# PCA - Not very good for categorical values
#pca_assembler = VectorAssembler(inputCols=df.columns, outputCol="pca_features",handleInvalid="skip")
#pca_df = pca_assembler.transform(df)
#print(pca_df)
#pca = PCA(k=5, inputCol="pca_features", outputCol="pcaFeatures")
#model = pca.fit(pca_df)

#result = model.transform(pca_features).select("pcaFeatures")
#result.show(truncate=False)

# 2. Preprocessing

In [ ]:
# Drop most data here since the data will be casted later therefor it will be considered string and not NaN
#df = df.drop('Number', 'Wind_Chill(F)', 'Precipitation(in)')
print(df.count())
#df.na.drop(*df.columns)

In [ ]:
print(df.count())

## Modify time

In [ ]:
# Convert to int then cast to string

df = df.withColumn('Start_Hour', hour(to_timestamp(from_utc_timestamp(df['Start_Time'], df['Timezone']).cast('string'), 'yyyy-MM-dd HH:mm:ss')))
df = df.withColumn('Start_Month', month(to_timestamp(from_utc_timestamp(df['Start_Time'], df['Timezone']).cast('string'), 'yyyy-MM-dd HH:mm:ss')))

df = df.withColumn('Weather_Hour', hour(to_timestamp(from_utc_timestamp(df['Weather_Timestamp'], df['Timezone']).cast('string'), 'yyyy-MM-dd HH:mm:ss')))
df = df.withColumn('Weather_Month', month(to_timestamp(from_utc_timestamp(df['Weather_Timestamp'], df['Timezone']).cast('string'), 'yyyy-MM-dd HH:mm:ss')))

df.show(1)

## Recheck the missing values

In [ ]:
df_missingVals_cols = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])
# Missing value in each column
df_missingVals_cols.collect()

## Cast data

In [ ]:
# Convert from string to integer values
# TODO: Does not work since data contains Nan/Null?
df = df.select(
        col('TMC').cast('string'),
        col('Severity').cast('int'),
        #col('Start_Time').cast('int'),
        col('Start_Hour').cast('string'),
        col('Start_Month').cast('string'),
        col('Weather_Hour').cast('string'),
        col('Weather_Month').cast('string'),
        #col('Start_Lat').cast('string'),
        #col('Start_Lng').cast('string'),
        col('Distance(mi)').cast('double'),
        col('Street').cast('string'),       # Remove eventually?
        col('Side').cast('string'),         # Remove eventually?
        col('City').cast('string'),         # Remove eventually?
        col('State').cast('string'),        # Remove eventually?
        #col('Zipcode').cast('string'),      # Remove eventually?
        #col('Country').cast('string'),      # Remove eventually?
        #col('Airport_Code').cast('string'), # Remove eventually?
      #  col('Weather_Timestamp').cast('int'),
        col('Temperature(F)').cast('double'),
        col('Wind_Chill(F)').cast('double'),
        col('Humidity(%)').cast('double'),
        col('Pressure(in)').cast('double'),
        col('Visibility(mi)').cast('double'),
        col('Wind_Direction').cast('string'),
        col('Wind_Speed(mph)').cast('double'),
        col('Weather_Condition').cast('string'),
        col('Amenity').cast('string'),
        col('Bump').cast('string'),
        col('Crossing').cast('string'),
        col('Give_Way').cast('string'),
        col('Junction').cast('string'),
        col('No_Exit').cast('string'),
        col('Railway').cast('string'),
        col('Roundabout').cast('string'),
        col('Station').cast('string'),
        col('Stop').cast('string'),
        col('Traffic_Calming').cast('string'),
        col('Traffic_Signal').cast('string'),
        col('Turning_Loop').cast('string'),
        col('Sunrise_Sunset').cast('string'),
        col('Civil_Twilight').cast('string'),
        col('Nautical_Twilight').cast('string'),
        col('Astronomical_Twilight').cast('string')
    ) 

# Since the last check can be unorganised we recreate a new list that contains all data
colLabel = ["Severity"]

rdd = sc.parallelize(df.dtypes)
colCat = rdd.map(lambda i: i[0] if (i[1]=='string' or i[1]=='boolean' and i[0] not in colLabel) else None).filter(lambda i: i != None).collect()
colNum = rdd.map(lambda i: i[0] if (i[1]=='double' and i[0] not in colLabel) else None).filter(lambda i: i != None).collect()

In [ ]:
print(colLabel)
print(colCat)
print(colNum)

### Quantiles function

In [ ]:
# Remove City, Country, State, Zipcode, Airport_Code
# In order to reduce dimensionality a kernel function will be applied upon the Start_Lat and Start_Lng then fused in order to create a new set of data
# Start_Lat
# Start_Lng

from pyspark.ml.feature import QuantileDiscretizer
"""
dscretizer_Lat = QuantileDiscretizer(numBuckets=50, inputCol="Start_Lat", outputCol="Start_Lat")
dscretizer_Lng = QuantileDiscretizer(numBuckets=50, inputCol="Start_Lng", outputCol="Start_Lng")

# Convert into categorical values (two blocks)
quant_Lat_model = quantiles_Lat.fit(df_features)
quant_Lng_model = quantiles_Lng.fit(df_features) 

quanted_Lat = quant_Lat_model.transform(df_features)
quanted_Lng = quant_Lng_model.transform(df_features)

# Into categorical values
stringIndexer = StringIndexer(inputCol=["Start_Lat","Start_Lng"], outputCol=["Start_Lat","Start_Lng"] + "_num")
indexer_model = stringIndexer.fit(df_features) # Change into Quanted_Lat, Quanted_Lng
indexed = indexer_model.transform(df_features)

# One-hot 
encoder = OneHotEncoder(inputCols=["Start_Lat_num","Start_Lng_num"], outputCols=[s + "_vec" for s in colCat])

encoded = encoder.fit(indexed)
"""

# Do multiplication into a final feature vector (Kernel trick)

# Remove City, Country, State, Zipcode, Airport_Code

### Impute 

In [ ]:
# Probobly a bad Idea but why not?
from pyspark.ml.feature import Imputer

#imputer = Imputer(inputCols=['Wind_Chill(F)', 'Wind_Speed(mph)'], outputCols=['Wind_Chill(F)', 'Wind_Speed(mph)'])
imputer = Imputer(inputCols=colNum, outputCols=colNum)
imputer.setStrategy("median")

### Scaling

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler

#Create a vector
num_assembler = VectorAssembler(inputCols=colNum, outputCol="num_features",handleInvalid="keep")
scaler = StandardScaler(inputCol="num_features", withMean=True, withStd=True, outputCol="scaledFeatures")
scaler

### Indexer

In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
#colCat
#col_assembler = VectorAssembler(inputCols=colCat, outputCol="cat_features",handleInvalid="keep")
#indexers = StringIndexer(inputCol="cat_features", outputCol="indexedFeatures",handleInvalid="keep")

indexers = [StringIndexer(inputCol = c, outputCol = c +'_IDX', handleInvalid='keep') for c in colCat]
print(indexers)

In [ ]:
# Debug
#from pyspark.ml import Pipeline
#pipeline = Pipeline(stages=indexers)
#df_indexers = pipeline.fit(df).transform(df)

#col_assembler = VectorAssembler(inputCols=[c +'_IDX' for c in colCat], outputCol="features",handleInvalid="keep")
#data = col_assembler.transform(df_indexers)
#df_features.withColumn('features', col(col_assembler.transform(df_indexers)))
#df_indexers.select([c +'_IDX' for c in colCat]).show(1)

### Encoder

In [ ]:
from pyspark.ml.feature import OneHotEncoder 

#ohe = OneHotEncoder()
#ohe.setInputCols([c + "_IDX" for c in colCat])
#ohe.setOutputCols([c + "_IDX_vec" for c in colCat])

#col_assembler = VectorAssembler(inputCols=[c + "_IDX" for c in colCat], outputCol="col_features",handleInvalid="keep")
encoder = OneHotEncoder(inputCols=[c + "_IDX" for c in colCat], outputCols=[c + "_IDX_vec" for c in colCat],handleInvalid="keep")
#encoder = OneHotEncoder(inputCol="col_features", outputCol="col_features_vec",handleInvalid="keep")
print(encoder)

In [ ]:
# Debug
#from pyspark.ml import Pipeline 
#pipeline = Pipeline(stages=[*indexers,*encoder])
#df_indexers = pipeline.fit(df).transform(df)
#df_indexers.select([c +'_IDX_vec' for c in colCat]).show(1)

In [ ]:
df.show(1)

## Preprocessing - Pipeline

In [ ]:
# Categorical values

from pyspark.ml import Pipeline

numPipeline = Pipeline(stages=[imputer, num_assembler, scaler])
catPipeline = Pipeline(stages=[*indexers, encoder])

#.select("scaledFeatures", *[c +'_IDX_vec' for c in colCat])
pipeline = Pipeline(stages=[numPipeline, catPipeline])
preprocessed_df = pipeline.fit(df).transform(df)


In [ ]:
catPipeline.fit(df).transform(df).show(1)

In [ ]:
preprocessed_df.select("scaledFeatures",*[c + "_IDX_vec" for c in colCat]).show(2,False)

## Combine features

In [ ]:
va2 = VectorAssembler(inputCols=["scaledFeatures",*[c + "_IDX_vec" for c in colCat]], outputCol="final_features")

temp1 = va2.transform(preprocessed_df)
temp1.show(1)
temp1 = temp1.withColumn('label', col("Severity"))
dataset = temp1.withColumn('features', temp1.final_features).select("features","label")
dataset.show(2, False)

# 3. Feature importance

In [ ]:
# ChiSqSelector
from pyspark.ml.feature import ChiSqSelector
selector = ChiSqSelector(numTopFeatures=3, featuresCol="features",
                         outputCol="selectedFeatures", labelCol="label")

result = selector.fit(dataset).transform(dataset)
print("ChiSqSelector output with top %d features selected" % selector.getNumTopFeatures())
result.show()

# 4. Machine learning

In [ ]:
trainSet, testSet = dataset.randomSplit([0.8 ,0.2], 1)

In [ ]:
trainSet.show(2,False)

In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(labelCol="label",maxIter=10, regParam=0.3, elasticNetParam=0.8)
lrModel = lr.fit(trainSet)
trainingSummary = lrModel.summary

print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))
print("RMSE: %s" % str(trainingSummary.rootMeanSquaredError))